In [1]:
import sys
import json
import torch
import sklearn
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from datetime import datetime

from torch.utils import data
from scipy.stats import spearmanr

# Our code
from data_generator import Dataset
from lstm import LSTM_model

In [2]:
train_file = "stab_data/stability_train.json"
valid_file = "stab_data/stability_valid.json"
test_file = "stab_data/stability_test.json"

In [3]:
class stab_dataset(Dataset):
    def __init__(self, filename, max_seq_len, output_type="onehot", acids="ACDEFGHIKLMNPQRSTVWY-"):
        elem_list = []
        label_list = []
        self.acids = acids
        self.output_type = output_type
        self.acid_dict, self.int_acid_dict = self.__gen_acid_dict__(acids)
        self.max_seq_len = max_seq_len
        self.get_prot_class=True
        # Loading the entire input file into memory
        for i, elem in enumerate(json.load(open(filename))):
            seq = elem["primary"].upper()
            if self.__is_legal_seq__(seq):
                elem_list.append(seq)
                label_list.append(elem["stability_score"][0])
        '''
        for i, elem in enumerate(SeqIO.parse(filename, "fasta")):
            if self.__is_legal_seq__(elem.seq.upper()):
                elem_list.append(elem.seq.upper())
                if get_prot_class:
                    label_list.append(prot_class_re.search(elem.description).group(1))
        '''
        
        self.data = elem_list
        self.prot_labels = label_list

    def __len__(self):
        return len(self.data)

class lin_reg(nn.Module):
    def __init__(self, input_size, output_size):
        super(lin_reg, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        #x = x.long()
        return self.linear(x)


In [4]:
use_cuda = torch.cuda.is_available() and True
print("Using GPU:", use_cuda)
processor = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

Using GPU: False
